In [1]:
'''from PIL import Image
path= 'C:/Users/51213/Desktop/9bus.png'
display(Image.open(path))'''
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
import math

model = AbstractModel()
model.t = Set(doc='set of time steps')
model.i = Set(doc='set of Gen units')
model.GBconnect = Set(dimen=2)
model.lines = Set(dimen=2)
model.bus = Set(doc='set of buses')
model.node = Set(initialize=model.bus)
model.b = Param(model.i, doc='b coef of cost', mutable=True)
model.pmin = Param(model.i, doc='Min gen', mutable=True)
model.pmax = Param(model.i, doc='Max gen', mutable=True)
model.dem = Param(model.t, doc='System demand', mutable=True)
model.sbase = Param(doc='Sbase', mutable=True)
model.branchX = Param(model.lines, doc='Max gen', mutable=True)
model.branchLim = Param(model.lines, doc='Max gen', mutable=True)
model.Pd = Param(model.bus, doc='Max gen', mutable=True)
# Determine the Pg.lo and Pg.up

def Pbounds(model,i,t):
    return (0*model.pmin[i]/model.sbase, model.pmax[i]/model.sbase)
# P bounds
model.P = Var(model.i, model.t, bounds=Pbounds, domain=Reals, doc='Generated thermal power')

def linelimit(model,bus,node,t):
    return (-model.branchLim[bus,node]/model.sbase,model.branchLim[bus,node]/model.sbase)
# Delta
model.delta = Var(model.bus  , model.t, bounds=(-math.pi, math.pi), domain=Reals, doc='angle')
model.flow  = Var(model.lines, model.t, bounds=linelimit,initialize=0, domain=Reals, doc='angle')

def NodesOut_init(model, bus):
    retval = []
    for (i,j) in model.lines:
        if i == bus and j != bus:
            retval.append(j)
    return retval
model.NodesOut = Set(model.bus, initialize=NodesOut_init,doc='outgoing flow nodes')

def Nodeswithgen(model, bus):
    retval = []
    for (gen,node) in model.GBconnect:
        if node == bus:
            retval.append(gen)
    return retval
model.GB = Set(model.bus, initialize=Nodeswithgen,doc='Nodes with generator')

def balance_rule(model,bus,t):
    return sum(model.P[i,t] for i in model.GB[bus]) - model.dem[t]*model.Pd[bus]/model.sbase == sum(model.flow[b,node,t] for (b,node) in model.lines if b==bus) 

def linecalc_rule(model,bus,node,t):
    if (bus,node) in model.lines:
        return model.flow[bus,node,t] == (1/model.branchX[bus,node])*(model.delta[bus,t] - model.delta[node,t])
    else:
        return Constraint.Skip
model.constflowcalc = Constraint(model.bus,model.node, model.t, rule=linecalc_rule)
model.constbalance = Constraint(model.bus, model.t , rule=balance_rule)
def objective_function_rule(model):
    return sum(model.b[i]*model.P[i,t] for i in model.i for t in model.t)
model.objective_function = Objective(rule=objective_function_rule, sense = minimize)

In [3]:
opt = SolverFactory('glpk')
instance = model.create_instance('powerflow-9bus.dat')
results = opt.solve(instance)
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  results.solver.status)

feasible


In [4]:
instance.constbalance.pprint()

constbalance : Size=216, Index=constbalance_index, Active=True
    Key        : Lower                  : Body                                             : Upper                  : Active
     ('b1', 1) :                    0.0 :    P[g1,1] - dem[1]*Pd[b1]/sbase - flow[b1,b4,1] :                    0.0 :   True
     ('b1', 2) :                    0.0 :    P[g1,2] - dem[2]*Pd[b1]/sbase - flow[b1,b4,2] :                    0.0 :   True
     ('b1', 3) :                    0.0 :    P[g1,3] - dem[3]*Pd[b1]/sbase - flow[b1,b4,3] :                    0.0 :   True
     ('b1', 4) :                    0.0 :    P[g1,4] - dem[4]*Pd[b1]/sbase - flow[b1,b4,4] :                    0.0 :   True
     ('b1', 5) :                    0.0 :    P[g1,5] - dem[5]*Pd[b1]/sbase - flow[b1,b4,5] :                    0.0 :   True
     ('b1', 6) :                    0.0 :    P[g1,6] - dem[6]*Pd[b1]/sbase - flow[b1,b4,6] :                    0.0 :   True
     ('b1', 7) :                    0.0 :    P[g1,7] - dem[7]*

In [5]:
print(value(instance.objective_function))
for i in instance.i:
    for t in instance.t:
        print(i,t, value(instance.P[i,t]) )

for L in instance.lines:
    for t in instance.t:
        print(L,t, value(instance.flow[L,t]))
        

3524.536900000001
g1 1 0.0
g1 2 0.0
g1 3 0.0
g1 4 0.0
g1 5 0.0
g1 6 0.0
g1 7 0.0
g1 8 0.0
g1 9 0.0
g1 10 0.0
g1 11 0.0
g1 12 0.040499999999998
g1 13 0.593
g1 14 0.643999999999998
g1 15 0.576000000000001
g1 16 0.507999999999999
g1 17 0.8735
g1 18 1.40900000000001
g1 19 1.324
g1 20 0.966999999999999
g1 21 0.7545
g1 22 0.397499999999997
g1 23 0.0320000000000036
g1 24 0.0
g2 1 2.5
g2 2 2.5
g2 3 2.5
g2 4 2.5
g2 5 2.5
g2 6 2.5
g2 7 2.5
g2 8 2.5
g2 9 2.5
g2 10 2.5
g2 11 2.5
g2 12 2.5
g2 13 2.5
g2 14 2.5
g2 15 2.5
g2 16 2.5
g2 17 2.5
g2 18 2.5
g2 19 2.5
g2 20 2.5
g2 21 2.5
g2 22 2.5
g2 23 2.5
g2 24 2.5
g3 1 2.3195
g3 2 2.022
g3 3 1.699
g3 4 1.41
g3 5 1.2315
g3 6 1.104
g3 7 1.155
g3 8 1.138
g3 9 1.155
g3 10 1.444
g3 11 2.0815
g3 12 2.5
g3 13 2.5
g3 14 2.5
g3 15 2.49999999999999
g3 16 2.5
g3 17 2.5
g3 18 2.5
g3 19 2.5
g3 20 2.50000000000001
g3 21 2.5
g3 22 2.5
g3 23 2.5
g3 24 2.396
('b1', 'b4') 1 0.0
('b1', 'b4') 2 0.0
('b1', 'b4') 3 0.0
('b1', 'b4') 4 0.0
('b1', 'b4') 5 0.0
('b1', 'b4') 6 -3.55

In [6]:
print('OF0= ', instance.objective_function.expr())

test = []
for i in instance.i:
    for t in instance.t:
        test.append([i , t, instance.P[i, t].value])
print(test)

test = []
for b in instance.bus:
    for t in instance.t:
        test.append([b , t, instance.delta[b, t].value])
print(test)

for v in instance.component_objects(pyo.Var, active=True):
    print("Variable",v)
    for index in v:
        print (" ",index, pyo.value(v[index]))

for v in instance.component_objects(pyo.Param, active=True):
    print("Parameter",v)
    for index in v:
        print (" ",index, pyo.value(v[index]))


#for (bus,node) in instance.lines
#    print(instance.flow[bus,node,'1'].value)


OF0=  3524.536900000001
[['g1', 1, 0.0], ['g1', 2, 0.0], ['g1', 3, 0.0], ['g1', 4, 0.0], ['g1', 5, 0.0], ['g1', 6, 0.0], ['g1', 7, 0.0], ['g1', 8, 0.0], ['g1', 9, 0.0], ['g1', 10, 0.0], ['g1', 11, 0.0], ['g1', 12, 0.040499999999998], ['g1', 13, 0.593], ['g1', 14, 0.643999999999998], ['g1', 15, 0.576000000000001], ['g1', 16, 0.507999999999999], ['g1', 17, 0.8735], ['g1', 18, 1.40900000000001], ['g1', 19, 1.324], ['g1', 20, 0.966999999999999], ['g1', 21, 0.7545], ['g1', 22, 0.397499999999997], ['g1', 23, 0.0320000000000036], ['g1', 24, 0.0], ['g2', 1, 2.5], ['g2', 2, 2.5], ['g2', 3, 2.5], ['g2', 4, 2.5], ['g2', 5, 2.5], ['g2', 6, 2.5], ['g2', 7, 2.5], ['g2', 8, 2.5], ['g2', 9, 2.5], ['g2', 10, 2.5], ['g2', 11, 2.5], ['g2', 12, 2.5], ['g2', 13, 2.5], ['g2', 14, 2.5], ['g2', 15, 2.5], ['g2', 16, 2.5], ['g2', 17, 2.5], ['g2', 18, 2.5], ['g2', 19, 2.5], ['g2', 20, 2.5], ['g2', 21, 2.5], ['g2', 22, 2.5], ['g2', 23, 2.5], ['g2', 24, 2.5], ['g3', 1, 2.3195], ['g3', 2, 2.022], ['g3', 3, 1.699], 

NameError: name 'pyo' is not defined